In [13]:
# Setup stuff

import google.generativeai as genai

# api key stuff

import os
genai.configure(api_key=os.getenv('GOOGLE_API_KEY'))

c:\Users\Sajiv\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# model
# using gemini-pro
# use gemini-pro-vision for text and images
model = genai.GenerativeModel('gemini-pro')
prompt = ""
response = model.generate_content("What is Google?")

In [4]:
response.text

'**Google LLC**\n\n**Overview:**\n\n* A multinational technology company specializing in Internet-related products and services.\n* Founded in 1998 by Larry Page and Sergey Brin.\n* Headquartered in Mountain View, California, United States.\n\n**Core Businesses:**\n\n**Search Engine:**\n\n* Google Search: Provides results for text, image, video, and other content on the web.\n* Google Maps: Provides detailed maps and navigation tools.\n* Google Earth: Delivers 3D representations of the Earth\'s surface.\n\n**Advertising:**\n\n* Google Ads: An advertising platform for businesses to reach targeted audiences.\n* Google AdSense: A program that allows website owners to earn revenue from displaying ads.\n\n**Cloud Computing:**\n\n* Google Cloud: Offers a wide range of cloud computing services, including storage, databases, and machine learning.\n\n**Software and Hardware:**\n\n* Android Operating System: Used in over 80% of smartphones worldwide.\n* Chrome Browser: A fast and secure web brow

In [2]:
import os

# Specify the path of the base directory
base_dir = 'kattis2'

# Get all entries in the directory
entries = os.listdir(base_dir)

# Filter out the entries to keep only directories
subfolders = [entry for entry in entries if os.path.isdir(os.path.join(base_dir, entry))]

# Count the subfolders
subfolder_count = len(subfolders)
print(f"Total subfolders: {subfolder_count}")

Total subfolders: 3705


In [46]:
def extract_code_to_file(code_block, filename='gemini.py'):
    # Split the block by lines
    lines = code_block.split('\n')
    # Remove the first and last lines which are the markdown code block markers
    stripped_lines = lines[1:-1]
    # Join the lines back into a single string
    code = '\n'.join(stripped_lines)
    
    # Write the extracted code to a file
    with open(filename, 'w', encoding='utf-8') as file:
        file.write(code)
    
    print(f"Code has been written to {filename}")

In [44]:
# trying out on the first 3 files

# model
# using gemini-pro
# use gemini-pro-vision for text and images
model = genai.GenerativeModel('gemini-pro')

responses = []

for subfolder in subfolders[:3]:

    print(subfolder)

    file_path = 'kattis2/' + subfolder
    # Open the file and read its contents into a string
    with open(file_path + '/prompt', 'r', encoding='utf-8') as file:
        prompt = file.read()

    response = model.generate_content("Answer the following in Python" + prompt)
    responses.append(response.text)
    extract_code_to_file(response.text, file_path + '/gemini.py')


10kindsofpeople
Code has been written to kattis2/10kindsofpeople/gemini.py
1dfroggereasy
Code has been written to kattis2/1dfroggereasy/gemini.py
1dfroggerhard
Code has been written to kattis2/1dfroggerhard/gemini.py


In [45]:
# get gemini code for prompts 4-55 (failed at 56) airrally

# model
# using gemini-pro
# use gemini-pro-vision for text and images
model = genai.GenerativeModel('gemini-pro')

responses = []

for subfolder in subfolders[4:100]:

    print(subfolder)

    file_path = 'kattis2/' + subfolder
    # Open the file and read its contents into a string
    with open(file_path + '/prompt', 'r', encoding='utf-8') as file:
        prompt = file.read()

    response = model.generate_content("Answer the following in Python" + prompt)
    responses.append(response.text)
    extract_code_to_file(response.text, file_path + '/gemini.py')


246greaaat
Code has been written to kattis2/246greaaat/gemini.py
2naire
Code has been written to kattis2/2naire/gemini.py
3dprinter
Code has been written to kattis2/3dprinter/gemini.py
3sideddice
Code has been written to kattis2/3sideddice/gemini.py
4thought
Code has been written to kattis2/4thought/gemini.py
8queens
Code has been written to kattis2/8queens/gemini.py
99problems
Code has been written to kattis2/99problems/gemini.py
a1paper
Code has been written to kattis2/a1paper/gemini.py
aaah
Code has been written to kattis2/aaah/gemini.py
abandonedanimal
Code has been written to kattis2/abandonedanimal/gemini.py
abcstring
Code has been written to kattis2/abcstring/gemini.py
abinitio
Code has been written to kattis2/abinitio/gemini.py
aboveaverage
Code has been written to kattis2/aboveaverage/gemini.py
abridgedreading
Code has been written to kattis2/abridgedreading/gemini.py
absolutelysymmetric
Code has been written to kattis2/absolutelysymmetric/gemini.py
abstractart
Code has been w

UnicodeEncodeError: 'charmap' codec can't encode characters in position 217-219: character maps to <undefined>

In [55]:
subfolders[56]

'airrally'

In [56]:
# get gemini code for prompts 4-99 ended on amsterdamdistance

# model
# using gemini-pro
# use gemini-pro-vision for text and images
model = genai.GenerativeModel('gemini-pro')


for subfolder in subfolders[56:100]:

    print(subfolder)

    file_path = 'kattis2/' + subfolder
    # Open the file and read its contents into a string
    with open(file_path + '/prompt', 'r', encoding='utf-8') as file:
        prompt = file.read()

    response = model.generate_content("Answer the following in Python" + prompt)
    extract_code_to_file(response.text, file_path + '/gemini.py')


airrally
Code has been written to kattis2/airrally/gemini.py
aladin
Code has been written to kattis2/aladin/gemini.py
alchemy101
Code has been written to kattis2/alchemy101/gemini.py
alchemy2
Code has been written to kattis2/alchemy2/gemini.py
alcohol
Code has been written to kattis2/alcohol/gemini.py
alea
Code has been written to kattis2/alea/gemini.py
alehouse
Code has been written to kattis2/alehouse/gemini.py
alexandbarb
Code has been written to kattis2/alexandbarb/gemini.py
alicedigital
Code has been written to kattis2/alicedigital/gemini.py
aliencodebreaking
Code has been written to kattis2/aliencodebreaking/gemini.py
alienintegers
Code has been written to kattis2/alienintegers/gemini.py
alienmath
Code has been written to kattis2/alienmath/gemini.py
alienmicrowave
Code has been written to kattis2/alienmicrowave/gemini.py
aliennumbers
Code has been written to kattis2/aliennumbers/gemini.py
aliens
Code has been written to kattis2/aliens/gemini.py
allaboutthatbase
Code has been writ

In [58]:
subfolders[99]

'amsterdamdistance'